In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
train_path = "/kaggle/input/home-data-for-ml-course/train.csv"
input_data = pd.read_csv(train_path)
input_data.head()

test_path = "/kaggle/input/home-data-for-ml-course/test.csv"
test_data = pd.read_csv(test_path)

In [3]:
y = input_data.SalePrice
X = input_data.iloc[:,:-1]

In [4]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [5]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [6]:
print(len(X.columns))
print(len(X.dropna(axis=1).columns))

80
61


In [7]:
#drop columns with Not Available values
X=X.dropna(axis=1)

print(len(test_data.columns))
test_data = test_data[X.columns]
print(len(test_data.columns))

80
61


In [8]:
# We need to do same for test data as well, it can contain na values as well
test_data = test_data.dropna(axis=1)
X = X[test_data.columns]

print(len(X.columns))

46


In [9]:
X.head()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1,60,8450,Pave,Reg,Lvl,Inside,Gtl,CollgCr,Norm,...,0,61,0,0,0,0,0,2,2008,Normal
1,2,20,9600,Pave,Reg,Lvl,FR2,Gtl,Veenker,Feedr,...,298,0,0,0,0,0,0,5,2007,Normal
2,3,60,11250,Pave,IR1,Lvl,Inside,Gtl,CollgCr,Norm,...,0,42,0,0,0,0,0,9,2008,Normal
3,4,70,9550,Pave,IR1,Lvl,Corner,Gtl,Crawfor,Norm,...,0,35,272,0,0,0,0,2,2006,Abnorml
4,5,60,14260,Pave,IR1,Lvl,FR2,Gtl,NoRidge,Norm,...,192,84,0,0,0,0,0,12,2008,Normal


In [10]:
test_data.head()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1461,20,11622,Pave,Reg,Lvl,Inside,Gtl,NAmes,Feedr,...,140,0,0,0,120,0,0,6,2010,Normal
1,1462,20,14267,Pave,IR1,Lvl,Corner,Gtl,NAmes,Norm,...,393,36,0,0,0,0,12500,6,2010,Normal
2,1463,60,13830,Pave,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,212,34,0,0,0,0,0,3,2010,Normal
3,1464,60,9978,Pave,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,360,36,0,0,0,0,0,6,2010,Normal
4,1465,120,5005,Pave,IR1,HLS,Inside,Gtl,StoneBr,Norm,...,0,82,0,0,144,0,0,1,2010,Normal


In [11]:
from sklearn.preprocessing import LabelEncoder

# create an empty dictionary to store the label encoder objects
label_encoder = {}

# get the categorical columns
categorical_columns = X.columns[X.dtypes != 'int64']

# apply the label encoder to the train data and store the objects
for col in categorical_columns:
    label_encoder[col] = LabelEncoder()
    X[col] = label_encoder[col].fit_transform(X[col])

# apply the same objects to transform the test data
for col in categorical_columns:
    test_data[col] = label_encoder[col].transform(test_data[col])


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
X.head()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1,60,8450,1,3,3,4,0,5,2,...,0,61,0,0,0,0,0,2,2008,4
1,2,20,9600,1,3,3,2,0,24,1,...,298,0,0,0,0,0,0,5,2007,4
2,3,60,11250,1,0,3,4,0,5,2,...,0,42,0,0,0,0,0,9,2008,4
3,4,70,9550,1,0,3,0,0,6,2,...,0,35,272,0,0,0,0,2,2006,0
4,5,60,14260,1,0,3,2,0,15,2,...,192,84,0,0,0,0,0,12,2008,4


In [13]:
test_data.head()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1461,20,11622,1,3,3,4,0,12,1,...,140,0,0,0,120,0,0,6,2010,4
1,1462,20,14267,1,0,3,0,0,12,2,...,393,36,0,0,0,0,12500,6,2010,4
2,1463,60,13830,1,0,3,4,0,8,2,...,212,34,0,0,0,0,0,3,2010,4
3,1464,60,9978,1,0,3,4,0,8,2,...,360,36,0,0,0,0,0,6,2010,4
4,1465,120,5005,1,0,1,4,0,22,2,...,0,82,0,0,144,0,0,1,2010,4


In [14]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=1)
model.fit(X,y)
predictions = model.predict(test_data)

In [15]:
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': predictions})
output.to_csv('submission.csv', index=False)